In [29]:
import pandas as pd
train_set = pd.read_json("./data/train_val_test/train_anonymized.json", orient="index")
valid_set = pd.read_json("./data/train_val_test/val_anonymized.json", orient="index")
test_set = pd.read_json("./data/train_val_test/test_anonymized.json", orient="index")
dataset_df = pd.read_json("./data/CovidET_anonymized.json", orient="index")
print (len(dataset_df))
print (len(train_set) + len(valid_set) + len(test_set))

1883
1883


In [30]:
# Eliminate examples where both annotators did "NA"
for idx, row in dataset_df.iterrows():
    anno_dict = row["Annotations"]
    i = 0
    for worker, work in anno_dict.items():
        for emo_dict in work:
            if emo_dict["Emotion"] == "NA":
                i += 1
    if i > 1:
        print (row)

In [31]:
import json
dataset_df = dataset_df.reset_index(drop=True)
final_json = dataset_df.to_json(orient="index")
results_json = json.loads(final_json)
with open("./data/CovidET_anonymized.json", 'w') as outfile:
    json.dump(results_json, outfile, indent=4)

In [32]:
print (len(train_set))
print (len(valid_set))
print (len(test_set))

1200
285
398


In [33]:
l1 = list(train_set["Reddit ID"]) + list(valid_set["Reddit ID"]) + list(test_set["Reddit ID"])
l2 = list(dataset_df["Reddit ID"])
res1 = [x for x in l1 if x not in l2]
res2 = [x for x in l2 if x not in l1]
print (res1, res2)

[] []


In [38]:
na_num = 0
for idx, row in dataset_df.iterrows():
    anno_dict = row["Annotations"]
    works = list(anno_dict.values())
    for work in works:
        if len(work) == 1 and work[0]["Emotion"] == "NA":
            na_num +=1
            break
print (na_num)

99


In [39]:
for idx, row in dataset_df.iterrows():
    anno_dict = row["Annotations"]
    works = list(anno_dict.values())
    if len(works[0]) == 1:
        if len(works[1]) == 1:
            if works[0][0]["Emotion"] == "NA":
                if works[1][0]["Emotion"] == "NA":
                    print (idx)

In [40]:
annotators = list()
for idx, row in dataset_df.iterrows():
    anno_dict = row["Annotations"]
    annotators.extend(key[76:] for key in list(anno_dict.keys()))
print (len(set(annotators)))

15


In [42]:
# if have duplicates
reddit_ids = list(dataset_df["Reddit ID"])
len(reddit_ids) == len(set(reddit_ids))

True

In [43]:
for idx, row in dataset_df.iterrows():
    anno_dict = row["Annotations"]
    if len(anno_dict) > 2:
        print (f"""Number of annotators error on {row["Reddit ID"]}""")

In [44]:
def rank_eval_dict(d: dict):
    eval_results = dict()
    for idx, key in enumerate(d):
        eval_results[f"{key[:11]}{idx}{key[12:]}"] = d[key]
    return eval_results

In [45]:
dataset_df.head()

,Reddit ID,Time Created,Annotations
0,o6lpwn,6/23/2021 20:51,{'Annotation 0 | Assignment ID = 3HMVI3QICT3FH...
1,o77vmk,6/24/2021 19:17,{'Annotation 0 | Assignment ID = 3HMVI3QICT3FH...
2,o78h66,6/24/2021 19:46,{'Annotation 0 | Assignment ID = 3B837J3LDY76E...
3,o7buf0,6/24/2021 23:42,{'Annotation 0 | Assignment ID = 3B837J3LDY76E...
4,o7fa6u,6/25/2021 3:15,{'Annotation 0 | Assignment ID = 35GMH2SV3OS27...


In [7]:
dataset_df = dataset_df.rename(columns={"Annotations": "old_Annotations"})
dataset_df = dataset_df.reset_index(drop=True)
dataset_df["Annotations"] = dataset_df['old_Annotations'].apply(rank_eval_dict)

In [8]:
import json
cleaned_json = dataset_df[["Reddit ID", "Time Created", "Reddit Post", "Annotations"]].to_json(orient="index")
results_json = json.loads(cleaned_json)
with open(f"../{dataset_name}.json", 'w') as outfile:
    json.dump(results_json, outfile, indent=4)